In [ ]:
#Import required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pymysql.cursors
import ast
import os,conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'Library'), 'share')
os.environ["PROJ_LIB"] = proj_lib
from mpl_toolkits.basemap import Basemap
from datetime import datetime
pd.options.mode.chained_assignment = None

#### Import chicago food inspection data taken from the below link.
https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/data

Description: 

This information is derived from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present. 

variables include:

DBA: "Doing Business as"

AKA: "Also known as"

License #: "License number"

Facility Type: Type of the facility like Restaurant, hospital, groceries, gas station etc..

Risk: Each establishment is categorized as Risk 1, 2, 3

Street address, city, state, zip code

Inspection type: Canvass, complaint, suspected food-poisoning

Results: pass, pass with conditions, fail, out of business, etc..

Violations: violation number, description of findings


In [ ]:
connection = pymysql.connect(host='localhost', user='root', password='root', db='food')

In [ ]:
#Test pandas read_sql
df = pd.read_sql('SELECT * from food_inspections', con = connection)
df.head(2)

In [ ]:
inspections_data = pd.read_csv("Food_Inspections.csv")

In [ ]:
print(inspections_data.shape)
inspections_data.head(2)

#### Make changes for the csv file so that the csv can be imported into sql table.

In [ ]:
sql_csv = inspections_data
sql_csv.drop(columns = ["Violations"], inplace = True)
sql_csv.to_csv("Food_Inspections_sql.csv", index = False)

#### Check for missing values in the data

In [ ]:
inspections_data.isnull().sum()

#### Here we have around 668 rows with no location. Let\`s delete them.

In [ ]:
inspections_data.dropna(subset = ['Location'], inplace = True)
print(len(inspections_data))

In [ ]:
inspections_data.isnull().sum()

#### Results column shows the result of the inspection. Inspections with values pass, pass with conditions or fail are of interest to us. We will delete the Inspections with other values in the results column.  

In [ ]:
print("All possible unique values of Results column:\n", inspections_data.Results.unique())
inspections_data = inspections_data[inspections_data.Results.isin(['Pass', 'Pass w/ Conditions', 'Fail'])]
#Around 22500 inspections were discarded.
print(len(inspections_data))

#### If there are any duplicate rows, drop them. 

In [ ]:
inspections_data.drop_duplicates(inplace=True)
print(len(inspections_data))

#### For our graphs/inferences to be more accurate, it is better to maintain consistency. So we change all values of some columns to uppercase. But this doesn\`t solve other problems in naming conventions(like space, special characters etc..) 

In [ ]:
inspections_data['DBA Name'] = inspections_data['DBA Name'].str.upper()
inspections_data['DBA Name'] = inspections_data['DBA Name'].str.replace("'", "")
inspections_data['Facility Type'] = inspections_data['Facility Type'].str.upper()


### 2)  Let\`s plot some bar graphs for variables in the data.

#### (i) Plot the number of inspections (frequency) for each result type 

In [ ]:
inspections_data.Results.value_counts().plot(kind = 'bar', title = 'Number of food inspections by result')
#So we have a lot of inspections which passed(around 1 lakh)

Hence, We have a lot of inspections with result pass.

#### (ii) Plot the number of inspections per month

In [ ]:
inspections_data['Inspection Date'] = pd.to_datetime(inspections_data['Inspection Date'])
inspections_per_month = inspections_data['Inspection Date'].dt.to_period('M').value_counts()
inspections_per_month.sort_index(inplace = True)
inspections_per_month.plot(kind = 'bar', figsize = (22, 12), title = "Number of food inspections per month")

The above plot shows the number of inspections per month from 2010 to 2018.

#### (iii) Plot the number of inspections for each risk type 

In [ ]:
inspections_data.Risk.value_counts().plot(kind = 'bar', title = 'Number of food inspections by risk')

Each establishment is categorized as to its risk of adversely affecting the public’s health, with 1 being the highest and 3 the lowest. The frequency of inspection is tied to this risk.

#### (iv) Plot the number of inspections by zipcode

In [ ]:
inspections_data.Zip.value_counts().plot(kind = 'bar', title = 'Number of food inspections by Zip', figsize = (18, 12))

#### (v) Plot the number of inspections for first 20 facility types with most inspections.

In [ ]:
inspections_data['Facility Type'].value_counts()[0:20].plot(kind = 'bar')

#### (vi) Plot the number of inspections for first 30 addresses with most inspections.

In [ ]:
inspections_data.Address.value_counts()[0:30].plot(kind = 'bar')

#### (vii) Plot the number of inspections for first 10 inspection types with most inspections.

In [ ]:
inspections_data['Inspection Type'].value_counts()[0:10].plot(kind = 'bar')

#### (viii) Top 20 food chains with most inspections.

In [ ]:
inspections_data['DBA Name'].value_counts()[0:20].plot(kind = 'bar')

In [ ]:
inspections_data[inspections_data.Address == '11601 W TOUHY AVE ']

#### Let\`s plot the location of all the food establishments where the inspection failed on chicago census tracts plot

In [ ]:
fig = plt.figure(figsize=[16,10])
m = Basemap(projection='merc',llcrnrlat=41.60,urcrnrlat=42.10,\
        llcrnrlon=-88.0,urcrnrlon=-87.50,lat_ts=20,resolution='c')
m.readshapefile('./geo_export_0e3da441-8fe8-4e11-9ca0-42ef75cab68e','chicago')

failed_inspections = inspections_data.loc[inspections_data.Results == 'Fail']

lat = []
lng = []
for coordinate in failed_inspections.Location:
    lat.append(ast.literal_eval(coordinate)[0])
    lng.append(ast.literal_eval(coordinate)[1])

x,y = m(lng,lat)
plt.scatter(x,y,alpha=0.5,s=20)
plt.title("Chicago Cenus Tracts")
plt.show()
    

#### Let\`s write some SQL Queries

In [ ]:
Failed_Facility_types = pd.read_sql('''SELECT Facility_Type, COUNT(Facility_Type) AS Total_Failed_Inspections FROM food_inspections WHERE Results = "Fail" GROUP BY Facility_Type ORDER BY 2 DESC LIMIT 5''', con = connection)

In [ ]:
Failed_Facility_types

#### Let\`s see how the number of inspections varied from 2016 to 2017

In [ ]:
inspections_inc = pd.read_sql('''SELECT (SELECT count(*) FROM food_inspections WHERE year(Inspection_Date) = 2016) - (SELECT count(*) FROM food_inspections WHERE year(Inspection_Date) = 2017) AS Decrease_in_Inspections_from_2016_to_2017 ''', con = connection)
inspections_inc

In [ ]:
failed_inspections_inc = pd.read_sql('''SELECT (SELECT count(*) FROM food_inspections WHERE year(Inspection_Date) = 2016 AND Results = "Fail") - (SELECT count(*) FROM food_inspections WHERE year(Inspection_Date) = 2017 AND Results = "Fail") AS Decrease_in_failedinspections_from_2016_to_2017 ''', con = connection)
failed_inspections_inc

#### Which places have maximum number of failed inspections?

In [ ]:
failed_places = pd.read_sql('''SELECT DBA_Name, Address, Location, count(*) AS Num_of_Failed_Inspections FROM food_inspections WHERE Results = "Fail" AND License != 0 GROUP BY License ORDER BY 4 DESC LIMIT 10''', con = connection)
failed_places

#### What is the percentage which didn\`t pass the inspections that were carried out because of complaints?

In [ ]:
df1 = pd.read_sql('''SELECT count(*)*(100.0) / (SELECT count(*) FROM food_inspections WHERE Inspection_Type  = "Complaint") AS Percentage FROM food_inspections WHERE Inspection_Type  = "Complaint" AND Results != "Pass"''', con = connection)
df1

#### What is the percentage which didn\`t pass the inspections that were carried out because of "Suspected Food Poisoning"?

In [ ]:
df2 = pd.read_sql('''SELECT count(*)*(100.0) / (SELECT count(*) FROM food_inspections WHERE Inspection_Type  = "Suspected Food Poisoning") AS Percentage FROM food_inspections WHERE Inspection_Type  = "Suspected Food Poisoning" AND Results != "Pass"''', con = connection)
df2

#### What is the percentage which didn\`t pass the inspections that were carried out because of "Complaint Re-Inspection"?

In [ ]:
df3 = pd.read_sql('''SELECT count(*)*(100.0) / (SELECT count(*) FROM food_inspections WHERE Inspection_Type = "Complaint Re-Inspection") AS Percentage FROM food_inspections WHERE Inspection_Type = "Complaint Re-Inspection" AND Results != "Pass"''', con = connection)
df3

#### What is the percentage which didn\`t pass the inspections that were carried out because of "Consultation"?

In [ ]:
df4 = pd.read_sql('''SELECT count(*)*(100.0) / (SELECT count(*) FROM food_inspections WHERE Inspection_Type = "Consultation") AS Percentage FROM food_inspections WHERE Inspection_Type = "Consultation" AND Results != "Pass"''', con = connection)
df4

#### Which Restaurants chains have most failed inspections?

In [ ]:
df5 = pd.read_sql('''SELECT DBA_Name, COUNT(*) as No_of_failed_inspections FROM food_inspections WHERE Results = "Fail" GROUP BY DBA_Name ORDER BY 2 DESC LIMIT 3''', con = connection)
df5

In [ ]:
inspections_data['DBA Name'][inspections_data.Results == "Fail"].value_counts()[0:5].plot(kind = 'bar', title = "Restaurant chains with most failed inspections")

#### Which subway restaurants failed most inspections?

In [ ]:
df6 = pd.read_sql('''SELECT License, Address, Zip, COUNT(License) AS No_of_failed_inspections FROM food_inspections WHERE DBA_Name = "SUBWAY" AND Results = "Fail" GROUP BY License ORDER BY 4 DESC LIMIT 5''', con = connection)
df6

#### Which individual restaurants failed most inspections?

In [ ]:
df7 = pd.read_sql('''SELECT DBA_Name, Address, Zip, COUNT(*) AS No_of_failed_inspections FROM food_inspections WHERE Facility_Type = "Restaurant" AND Results = "Fail" AND License != 0 GROUP BY License ORDER BY 4 DESC LIMIT 5''', con = connection)
df7

In [ ]:
plt.bar(df7['DBA_Name'], df7['No_of_failed_inspections'])
plt.xticks(rotation = "vertical")
plt.title("Top 5 restaurants with most failed inspections")
plt.show()

#### Addresses with most failed inspections

In [ ]:
df8 = pd.read_sql('''SELECT Address, COUNT(*) AS No_of_failed_inspections FROM food_inspections WHERE Results = "Fail" GROUP BY Address ORDER BY 2 DESC LIMIT 5''', con = connection)
df8

In [ ]:
plt.bar(df8['Address'], df7['No_of_failed_inspections'])
plt.xticks(rotation = "vertical")
plt.title("Top 5 Addresses with most failed inspections")
plt.show()